In [1]:

from pathlib import Path, PosixPath

print(Path.home())
import pandas.plotting
import mysql.connector # pip install mysql-connector-python-rf
import sys
print(sys.version)
import tensorflow as tf 
#import git # pip install GitPython
import os
import matplotlib.pyplot as plt # pip install matplotlib
import numpy as np
import time
import pandas as pd 

/home/og016
3.7.3 (default, Apr  3 2019, 05:39:12) 
[GCC 8.3.0]


In [2]:
print("=====>GPU Available: ", tf.test.is_gpu_available())
tf.debugging.set_log_device_placement(True)
tf.config.experimental.list_physical_devices()
#tf.config.gpu_options.allow_growth = True 
#tf.config.experimental.set_memory_growth( True)
print(tf.version.GIT_VERSION, tf.version.VERSION)

=====>GPU Available:  True


AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'gpu_options'

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print("=====>", len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

## Config

In [ ]:
#Log Data
print('Modification:')
modification = input() # Was wurde verändert
print('User:')
user = input()#git.util.get_user_id() # Bearbeiter, Sollte automatisch gelesen werden 
try:
    repo = git.Repo()
    branch = repo.active_branch # Projekt, Sollte automatisch gelesen werden
    lastCommit = repo.head.commit.hexsha #"ca324dadsa" # Stand, Sollte automatisch gelesen werden
except:
    print('Branch:')
    
    branch = input()
    lastCommit = ''
env = "local" # Test umgebung

In [ ]:
# Model Config
samplesize = 1000 # Datensatz
split = 0.8  # Testdatensatz in % => 0.8 = 80% testdata
BATCH_SIZE = 20
IMG_HEIGHT = 224
IMG_WIDTH = 224
AUTOTUNE = tf.data.experimental.AUTOTUNE

## Load Images

In [ ]:
data_dir = Path(os.path.join(Path.home(), 'Pictures/Pictures_Bicycles'))
image_count = len(list(data_dir.glob('*/*.jpg')))
CLASS_NAMES = []
for f in np.array([item.name for item in data_dir.glob('Training/*')]):
    if f.startswith('uni'):
        name = f.split('_')[0]
    else:
        name = f.split('_')[1]
    if name not in CLASS_NAMES :
        CLASS_NAMES.append(name)
print(CLASS_NAMES)
print(image_count)

def showBatch(image_batch, label_batch, prediction=None):
    plt.figure(figsize=(10,10))
    for n in range(10):
        ax = plt.subplot(5,5,n+1)
        ax.imshow(image_batch[n])
        if label_batch is not None:
            plt.title(CLASS_NAMES[label_batch[n]])
        if prediction is not None:
            plt.title(CLASS_NAMES[np.argmax(prediction[n])])
        ax.axis('off')
    plt.show()
        
def getLabel(path):
    # geht class name from last folder
    #folder = os.path.basename(os.path.dirname(path))
    # gets the index of the classname
    #return tf.convert_to_tensor(np.where(CLASS_NAMES == folder)[0][0], dtype=tf.int32)
    path = Path(path).stem
    if str(path).startswith('uni'):
        folder = str(path).split('_')[0]
    else:
        folder = str(path).split('_')[1]
        
    return tf.convert_to_tensor(CLASS_NAMES.index(folder), dtype=tf.int32)
        
def loadImg(path):
    try:
        img = tf.io.read_file(str(path))
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    except:
        return path
    


In [ ]:
def loadDataset(role='train'):
    #list_ds = tf.data.Dataset.list_files(os.path.join(str(data_dir),'*','*.jpg'))
    #ds = list_ds.map(loadImg, num_parallel_calls=AUTOTUNE)
    data = Path(os.path.join(data_dir, role)) 
    paths = list(data.glob('*.jpg'))
    #myImage.loadData(pfad)
    #remove false jpg's
    images = []
    for p in paths:
        out = loadImg(p)
        if type(out) == PosixPath:
            paths.remove(out)
        else:
            images.append(out)
    #images = list(map(loadImg, paths))
    labels = list(map(getLabel, paths))
    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    ##Prepare for Training 
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.repeat()
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds
train_ds = loadDataset('Training')
#validate_ds = loadDataset('validate')
test_ds = loadDataset('Test')

## TF Code

In [ ]:
### Your Code
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.summary()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
## Start Timer 
start = time.time()
val_max_acc = 2 # amount of loops with max acc til stop 

for n in range(5):
    img_batch, label_batch = next(iter(train_ds))
    model.fit(img_batch, label_batch, epochs=10)


    timg_batch, tlabel_batch = next(iter(validate_ds))
    test_loss, test_acc = model.evaluate(timg_batch,  tlabel_batch, verbose=2)
    
## End Timer
end = time.time()
runtime = end-start

print('\nTest accuracy: {}, \nRuntime: {}'.format(test_acc, runtime))


images, rlabel_batch = next(iter(test_ds))
predictions = model.predict(images)
showBatch(images.numpy(), None, predictions)

In [1]:
runtime = 10 # Laufzeit aus tf
maxAcc = 0.85 # Acc. aus tf
maxLoss = 0.2 # Loss aus tf

## Log